In [1]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/home/dmpowell/.conda/envs/pytorch-pip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device =  cuda


In [2]:
MODEL_NAME = "EleutherAI/gpt-j-6B" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = GPTJForCausalLM.from_pretrained("databricks/dolly-v1-6b", torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [33]:
def generate_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=True, temperature=0.7, max_new_tokens=150)

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)

texts = ["Based on your extensive knowledge, list 5 of the most important facts about Jimi Hendrix. List each fact on its own line and start each fact with 'Jimi Hendrix'.\n"]
for t in generate_text(texts, model): print(t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Based on your extensive knowledge, list 5 of the most important facts about Jimi Hendrix. List each fact on its own line and start each fact with 'Jimi Hendrix'.

1. He is best known for his guitar playing.
2. He is the only musician to have been inducted into the Rock and Roll Hall of Fame twice.
3. He was a huge influence on many artists, especially from the 1960s.
4. He had an extensive solo career, including several Grammy awards.
5. He is often referred to as a pioneer of electric guitar.

1. Hendrix was born James Marshall Hendrix in 1942 in Seattle, Washington.
2. He was a blues and jazz guitarist, and he was influenced by the music of Buddy Guy, Albert King, and John Lee Hooker.
3. He began playing the guitar at the age of seven, and at the
